In [ ]:
import os
import openai
from openai.error import RateLimitError, APIConnectionError, InvalidRequestError
import pandas as pd
import time

In [ ]:
#pip install openai==0.28

#### GPT-4 configuration

In [ ]:
azure_endpoint = "https://rumi-gpt4.openai.azure.com/"
api_key = "45d17eeab9904917a8ba16c282670301"
api_version = "2024-02-15-preview"
openai.api_type = "azure"
openai.api_base = azure_endpoint
openai.api_key = api_key
openai.api_version = api_version

#### GPT-3.5 configuration

In [ ]:
azure_endpoint = "https://rumi-v0.openai.azure.com/"
api_key = "bb40601a473b4e25ad9c3975bd3c47ce"
api_version = "2023-07-01-preview"
openai.api_type = "azure"
openai.api_base = azure_endpoint
openai.api_key = api_key
openai.api_version = api_version

In [ ]:
def load_prompt(review):
    # Read the prompt
    with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/prompt_for_decomposing_review_restaurants.txt', 'r') as file:
    #with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/prompt_for_decomposing_review_hairsalons.txt', 'r') as file:
        prompt = file.read()
        
        # Read the few-shot examples
        with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/few_shot_for_decomposing_review_restaurants.txt', 'r') as fewshot_file:
        #with open('/Users/innerpiece92/Desktop/NLP_Workspace/AArec/prompt/extracting/few_shot_for_decomposing_review_hairsalons.txt', 'r') as fewshot_file:
            few_shot_examples = fewshot_file.read()
        
        prompt = prompt.format(review=review, few_shot_examples=few_shot_examples)
        #print(prompt)

    return prompt

In [ ]:
reviews_w_AtA = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/restaurants/test/rcb_using_decomposed_review_classification_v3_gpt35/restaurant_reviews_F.xlsx'
#reviews_w_AtA = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hotels/dev/dev_w_ids.xlsx'
#reviews_w_AtA = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hair_salons/dev/dev_w_ids.xlsx'
reviews_w_AtA_df = pd.read_excel(reviews_w_AtA)
reviews_w_AtA_df.head(10)

### GPT-4

In [ ]:
data_to_process = []

for review_index, review_row in reviews_w_AtA_df.iterrows():
    review = review_row['review']
    name = review_row['name']
    business_id = review_row['business_id']
    data_to_process.append((name, business_id, review))
    
results = []

for name, business_id, review in data_to_process:
    try:
        
        prompt = load_prompt(review)
        
        messages = [
              {"role": "system", "content": "You are a helpful AI assistant"},
              {"role": "user", "content": prompt}
        ]
        
        formatted_messages = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
        print("Messages being sent to GPT:\n", formatted_messages)

        # Use OpenAI's chat completion endpoint 
        response = openai.ChatCompletion.create(
            messages=messages,
            engine="gpt-4",
            temperature=0,
            frequency_penalty=0,
            presence_penalty=0,
            max_tokens=500
        )
        
        # Append the result to the list
        result_content = response['choices'][0]['message']['content']
        results.append((name, business_id, review, result_content))
        print(results)


    except openai.error.InvalidRequestError as e:
        print(f"Skipping due to error: {e}")
    except RateLimitError as e:
        print("Rate limit exceeded. Retrying in 150 seconds...")
        time.sleep(150)
    except APIConnectionError as e:
        print("APIConnectionError: Failed to connect to the API. Please check your network connection and API configuration:", e)
    except Exception as e:
        print("An unexpected error occurred:", e)

### GPT-3.5

In [ ]:
data_to_process = []

for review_index, review_row in reviews_w_AtA_df.iterrows():
    review = review_row['review']
    name = review_row['name']
    business_id = review_row['business_id']
    data_to_process.append((name, business_id, review))
    
results = []

for name, business_id, review in data_to_process:
    try:
        
        prompt = load_prompt(review)
        
        messages = [
              {"role": "system", "content": "You are a helpful AI assistant"},
              {"role": "user", "content": prompt}
        ]
        
        formatted_messages = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
        print("Messages being sent to GPT:\n", formatted_messages)

        # Use OpenAI's chat completion endpoint 
        response = openai.ChatCompletion.create(
            messages=messages,
            engine="gpt-35-turbo",
            temperature=0,
            frequency_penalty=0,
            presence_penalty=0,
            max_tokens=500
        )
        
        # Append the result to the list
        result_content = response['choices'][0]['message']['content']
        results.append((name, business_id, review, result_content))
        print(results)


    except openai.error.InvalidRequestError as e:
        print(f"Skipping due to error: {e}")
    except RateLimitError as e:
        print("Rate limit exceeded. Retrying in 150 seconds...")
        time.sleep(150)
    except APIConnectionError as e:
        print("APIConnectionError: Failed to connect to the API. Please check your network connection and API configuration:", e)
    except Exception as e:
        print("An unexpected error occurred:", e)

In [ ]:
data_to_process = []

for review_index, review_row in reviews_w_AtA_df.iterrows():
    review = review_row['review']
    name = review_row['name']
    business_id = review_row['business_id']
    data_to_process.append((name, business_id, review))
    
results = []

for name, business_id, review in data_to_process:
    #try:
        
    prompt = load_prompt(review)

    messages = [
          {"role": "system", "content": "You are a helpful AI assistant"},
          {"role": "user", "content": prompt}
    ]

    formatted_messages = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
    print("Messages being sent to GPT:\n", formatted_messages)

    # Use OpenAI's chat completion endpoint 
    response = openai.ChatCompletion.create(
        messages=messages,
        engine="gpt-4o-tonmoy",
        temperature=0,
        frequency_penalty=0,
        presence_penalty=0,
        max_tokens=500
    )

    # Append the result to the list
    result_content = response['choices'][0]['message']['content']
    results.append((name, business_id, review, result_content))
    print(results)


    '''except openai.error.InvalidRequestError as e:
        print(f"Skipping due to error: {e}")
    except RateLimitError as e:
        print("Rate limit exceeded. Retrying in 150 seconds...")
        time.sleep(150)
    except APIConnectionError as e:
        print("APIConnectionError: Failed to connect to the API. Please check your network connection and API configuration:", e)
    except Exception as e:
        print("An unexpected error occurred:", e)'''

In [ ]:
corrected_results = []
for r in results:
    if len(r) == 4:
        name, business_id, review, decomposed_review = r
        corrected_results.append((name, business_id, review, decomposed_review))
    else:
        print(f"Skipping incorrect entry: {r}")

# Create a DataFrame
results_df = pd.DataFrame(corrected_results, columns=['name', 'business_id', 'review', 'decomposed_review'])

# Define output file path
output_file = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/restaurants/test/rcb_using_decomposed_review_classification_v3_gpt35/train_test_w_ids_decomposed.xlsx'
#output_file = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hotels/dev/dev_w_ids_decomposed.xlsx'
#output_file = '/Users/innerpiece92/Desktop/NLP_Workspace/AArec/datasets/extracting_atypical_aspects_of_items_from_reviews/hair_salons/dev/dev_w_ids_decomposed.xlsx'

# Save the DataFrame to an Excel file
results_df.to_excel(output_file, index=False)

print(f"Data successfully saved to {output_file}")